In [1]:
#https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

# Importing the dataset
import pandas as pd
from sklearn.model_selection import KFold

dataset = pd.read_csv('/home/left/Downloads/spambase.data',header = None)
# Changing pandas dataframe to numpy array
X = dataset.iloc[:, 0:57].values # We have 58 columns
y = dataset.iloc[:, 57].values # Set the output column

# Normalizing the data 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
Χ = sc.transform(X)

input_train = []
input_test = []
output_train = []
output_test = []

# 10-fold cross-validation
kf = KFold(n_splits = 10,shuffle = True)

for train_index, test_index in kf.split(dataset):
    
    input_train.append(X[train_index])
    input_test.append(X[test_index])
    output_train.append(y[train_index])
    output_test.append(y[test_index])


In [15]:
#https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

# Importing the dataset
import pandas as pd
from sklearn.model_selection import KFold

dataset = pd.read_excel('/home/left/Downloads/default_of_credit_card_clients.xls',header = None)
# Changing pandas dataframe to numpy array
X = dataset.iloc[2:, 1:24].values
y = dataset.iloc[2:, 24].values
dataset = dataset.drop(columns=[0])
dataset = dataset.drop([0, 1])

# Normalizing the data 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
Χ = sc.transform(X)

input_train = []
input_test = []
output_train = []
output_test = []

# 10-fold cross-validation
kf = KFold(n_splits = 10,shuffle = True)

for train_index, test_index in kf.split(dataset):
    
    input_train.append(X[train_index])
    input_test.append(X[test_index])
    output_train.append(y[train_index].astype('int'))
    output_test.append(y[test_index].astype('int'))


In [2]:
# Importing the libraries
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

average_acc = 0
average_f1 = 0 

for i in range(10):
    
    classifier = KNeighborsClassifier(n_neighbors = 25, p=2) #Euclidean distance
    classifier = classifier.fit(input_train[i],output_train[i])
    
    y_pred = classifier.predict(input_test[i])
    
    average_acc += accuracy_score(output_test[i], y_pred)
    average_f1 += f1_score(output_test[i],y_pred)

print("The accuracy is", average_acc/10)
print("The F1 Score is", average_f1/10)

The accuracy is 0.8989389795340943
The F1 Score is 0.8644900912553191


In [4]:
#LVQ
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import math
from scipy.spatial import distance
# global variables
alpha = 0.001
acc = 0
f1 = 0
total_acc = []
total_f1 = []
first_class = []
zero_class = []
output_class = [] # y


# separate all inputs base on their classes 
for i in range(10):
    for j in range(len(output_train[i])):
        if output_train[i][j] == 1:
            first_class.append(input_train[i][j])
            output_class.append(output_train[i][j])
        elif output_train[i][j] == 0:
            zero_class.append(input_train[i][j])
            output_class.append(output_train[i][j])

zero_class_mean = sum(zero_class) / len(zero_class) # init mean for first region (class 0)
first_class_mean = sum(first_class) / len(first_class) # init mean for second region (class 1)

first_std = np.std(first_class) # init std for first region
zero_std = np.std(zero_class) # init std for second region

centroids = {'mean':[zero_class_mean], 'std':[zero_std], 'Cj':[0]} # init first two centroids for each class
centroids['mean'].append(first_class_mean) # init centroid for class 1
centroids['std'].append(first_std)
centroids['Cj'].append(1)

input_x = first_class + zero_class # all input vectors

def gaussian_similarity(input_x, max_similarity, pred):
    
    if pred==0:
        for m in range(len(centroids['mean'])):
            numerator = np.sum(np.power(input_x - centroids['mean'][m], 2))
            temp = numerator/(2*(centroids['std'][m]**2))
            gaussian = math.exp(-temp)

            if (gaussian > max_similarity):
                max_similarity = gaussian
                winner_index = m
        return winner_index

    for m in range(len(centroids['mean'])):
        numerator = np.sum(np.power(input_test[i][j] - centroids['mean'][m], 2))
        temp = numerator/(2*(centroids['std'][m]**2))
        gaussian = math.exp(-temp)
        
        if (gaussian > max_similarity):
            max_similarity = gaussian
            winner_index = m
            y_pred = centroids['Cj'][winner_index]
    return y_pred
    
# train the model 
for i in range(len(input_x)):

    winner_index = gaussian_similarity(input_x[i], max_similarity = 0, pred = 0)

    # success
    if (output_class[i] == centroids['Cj'][winner_index]):
        centroids['std'][winner_index] = centroids['std'][winner_index] + alpha*(distance.euclidean(input_x[i], centroids['mean'][winner_index]))
        centroids['mean'][winner_index] = (1-alpha)*centroids['mean'][winner_index] + alpha*input_x[i]
    # failure
    else:
        centroids['std'][winner_index] = centroids['std'][winner_index] - alpha*(distance.euclidean(input_x[i], centroids['mean'][winner_index]))
        centroids['mean'][winner_index] = (1+alpha)*centroids['mean'][winner_index] - alpha*input_x[i]
        # create new centroid
        centroids['mean'].append(input_x[i])
        centroids['std'].append(0.1*np.std(input_x))
        centroids['Cj'].append(centroids['Cj'][winner_index])

print('training finished')
# make predictions and testing
for i in range(10):
    for j in range(len(output_test[i])):
        y_pred = gaussian_similarity(input_test[i][j], max_similarity = 0, pred = 1)           
        total_acc.append(y_pred)
        total_f1.append(y_pred)

    acc += accuracy_score(output_test[i], total_acc)
    f1 += f1_score(output_test[i], total_acc)
    total_f1.clear()
    total_acc.clear()
        
print('The accuracy is: ', acc/10)
print('The F1 score is: ', f1/10)


training finished
The accuracy is:  0.5868070357446007
The F1 score is:  0.6568923535834693


In [17]:
#MLP
#https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense
#https://www.programcreek.com/python/example/89667/keras.layers.Dense
#https://keras.io/models/model/
#https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/
#importing the libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

average_acc = 0
average_f1 = 0

for i in range(10):
    
    nn = tf.keras.Sequential()

    nn.add(layers.Dense(50, input_dim = 57, activation = 'sigmoid')) #densely-connected 1st layer and 1st hidden layer

    nn.add(layers.Dense(70, activation = 'sigmoid')) #second hidden layer with 15 units

    nn.add(layers.Dense(1, activation = 'sigmoid')) #output layer
    
    nn.compile(optimizer='adam', loss = 'mean_squared_error', metrics = ['accuracy'])
    
    nn.fit(input_train[i], output_train[i], epochs=20, batch_size=32)
    
    # Performance on test data
    # If the prediction is greater than 0.5 then the output is 1 else the output is 0
    y_pred = nn.predict(input_test[i])
    y_pred = (y_pred > 0.5)
            
    average_acc += accuracy_score(output_test[i], y_pred)
    average_f1 += f1_score(output_test[i], y_pred)

print("The accuracy is", average_acc/10)
print("The F1 Score is", average_f1/10)



Train on 4140 samples
Epoch 1/20
4140/4140 [==============================] - 1s 322us/sample - loss: 0.1957 - accuracy: 0.7070
Epoch 2/20
4140/4140 [==============================] - 1s 203us/sample - loss: 0.0888 - accuracy: 0.9012
Epoch 3/20
4140/4140 [==============================] - 1s 186us/sample - loss: 0.0692 - accuracy: 0.9186
Epoch 4/20
4140/4140 [==============================] - 1s 156us/sample - loss: 0.0623 - accuracy: 0.9244
Epoch 5/20
4140/4140 [==============================] - 1s 193us/sample - loss: 0.0583 - accuracy: 0.9314
Epoch 6/20
4140/4140 [==============================] - 1s 185us/sample - loss: 0.0558 - accuracy: 0.9355
Epoch 7/20
4140/4140 [==============================] - 1s 207us/sample - loss: 0.0544 - accuracy: 0.9350
Epoch 8/20
4140/4140 [==============================] - 1s 161us/sample - loss: 0.0531 - accuracy: 0.9379
Epoch 9/20
4140/4140 [==============================] - 1s 182us/sample - loss: 0.0524 - accuracy: 0.9379
Epoch 10/20
4140/4140 [=

4141/4141 [==============================] - 1s 167us/sample - loss: 0.0464 - accuracy: 0.9418
Epoch 18/20
4141/4141 [==============================] - 1s 153us/sample - loss: 0.0457 - accuracy: 0.9457
Epoch 19/20
4141/4141 [==============================] - 1s 157us/sample - loss: 0.0454 - accuracy: 0.9445
Epoch 20/20
4141/4141 [==============================] - 1s 194us/sample - loss: 0.0449 - accuracy: 0.9466
Train on 4141 samples
Epoch 1/20
4141/4141 [==============================] - 1s 257us/sample - loss: 0.1674 - accuracy: 0.7588
Epoch 2/20
4141/4141 [==============================] - 1s 211us/sample - loss: 0.0796 - accuracy: 0.9123
Epoch 3/20
4141/4141 [==============================] - 1s 160us/sample - loss: 0.0663 - accuracy: 0.9222
Epoch 4/20
4141/4141 [==============================] - 1s 191us/sample - loss: 0.0606 - accuracy: 0.9268
Epoch 5/20
4141/4141 [==============================] - 1s 146us/sample - loss: 0.0577 - accuracy: 0.9326
Epoch 6/20
4141/4141 [==========

In [3]:
#SVM_linear
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.svm import SVC

average_acc = 0
average_f1 = 0

for i in range(10):
    
    svm = SVC(kernel='linear', gamma='auto') # Linear Kernel
    svm = svm.fit(input_train[i],output_train[i])
    
    y_pred = svm.predict(input_test[i])
    
    average_acc += accuracy_score(output_test[i], y_pred)
    average_f1 += f1_score(output_test[i],y_pred)

print("The accuracy is")
print(average_acc/10)

print("The F1 Score is")
print(average_f1/10)



The accuracy is
0.9284914646798075
The F1 Score is
0.9075703410081364


In [4]:
#SVM_gaussian
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.svm import SVC

average_acc = 0
average_f1 = 0

for i in range(10):
    
    svm = SVC(kernel='rbf', gamma='auto') # Gaussian Kernel
    svm = svm.fit(input_train[i],output_train[i])
    
    y_pred = svm.predict(input_test[i])
    average_acc += accuracy_score(output_test[i], y_pred)
    average_f1 += f1_score(output_test[i],y_pred)

print("The accuracy is")
print(average_acc/10)

print("The F1 Score is")
print(average_f1/10)

The accuracy is
0.9334891068565501
The F1 Score is
0.9136590583366064


In [5]:
#NaiveBayes
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB
 
average_acc = 0
average_f1 = 0
    
for i in range(10):
    
    nb = GaussianNB()
    nb = nb.fit(input_train[i],output_train[i])
    
    y_pred = nb.predict(input_test[i])
    
    average_acc += accuracy_score(output_test[i], y_pred)
    average_f1 += f1_score(output_test[i],y_pred)

print("The accuracy is", average_acc/10)
print("The F1 Score is", average_f1/10)



The accuracy is 0.8183061397717626
The F1 Score is 0.8050742389388716
